# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [1]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Import API key
from api_keys import geoapify_key

In [2]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,port-aux-francais,-49.3500,70.2167,43.16,96,100,32.30,TF,1717872420
1,1,weno,7.4515,151.8468,82.35,83,75,4.61,FM,1717872420
2,2,enewetak,11.3474,162.3373,82.69,81,31,15.01,MH,1717872420
3,3,salekhard,66.5300,66.6019,33.40,80,24,7.00,RU,1717872420
4,4,tiksi,71.6872,128.8694,32.09,90,57,8.79,RU,1717872420


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [3]:

%%capture --no-display

humidity = city_data_df["Humidity"].astype(int)

# Configure the map plot
map_plot_1 = city_data_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    frame_width = 800,
    frame_height = 600,
    size = "Humidity",
    color="City",
    alpha=.75
)

# Display the map
map_plot_1

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [4]:
# Narrow down cities that fit criteria and drop any results with null values
city_data_df = city_data_df.loc [(city_data_df["Max Temp"] > 30)]

city_data_df = city_data_df.loc[(city_data_df["Wind Speed"] < 4.5)]

city_data_df = city_data_df.loc[(city_data_df["Cloudiness"] == 0)]

# Drop any rows with null values
city_data_df = city_data_df.dropna()

# Display sample data
city_data_df

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
32,32,hohhot,40.8106,111.6522,57.49,72,0,2.24,CN,1717872421
33,33,bredasdorp,-34.5322,20.0403,50.34,86,0,2.37,ZA,1717872421
42,42,port elizabeth,-33.9180,25.5701,55.69,94,0,3.44,ZA,1717872422
101,101,jampur,29.6441,70.5954,93.25,27,0,2.95,PK,1717872424
119,119,nelson bay,-32.7167,152.1500,57.09,87,0,1.01,AU,1717872425
128,128,bilibino,68.0546,166.4372,43.30,48,0,3.36,RU,1717872356
161,161,sanjwal,33.7608,72.4336,81.68,31,0,4.29,PK,1717872427
187,187,kingscliff,-28.2667,153.5667,51.73,100,0,1.14,AU,1717872428
223,223,romorantin-lanthenay,47.4167,1.8333,67.66,73,0,0.00,FR,1717872430
248,248,sur,22.5667,59.5289,88.52,58,0,4.07,OM,1717872431


### Step 3: Create a new DataFrame called `hotel_df`.

In [5]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df=city_data_df[["City", "Country", "Lat", "Lng","Humidity"]].copy()
# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df["Hotel Name"]= ""
# Display sample data
hotel_df

,City,Country,Lat,Lng,Humidity,Hotel Name
32,hohhot,CN,40.8106,111.6522,72,
33,bredasdorp,ZA,-34.5322,20.0403,86,
42,port elizabeth,ZA,-33.9180,25.5701,94,
101,jampur,PK,29.6441,70.5954,27,
119,nelson bay,AU,-32.7167,152.1500,87,
128,bilibino,RU,68.0546,166.4372,48,
161,sanjwal,PK,33.7608,72.4336,31,
187,kingscliff,AU,-28.2667,153.5667,100,
223,romorantin-lanthenay,FR,47.4167,1.8333,73,
248,sur,OM,22.5667,59.5289,58,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [6]:
radius = 10000
categories = "accommodation.hotel"
limit = 20

# Set parameters for type of search
params = {"categories": categories,
          "limit": limit,
          "apiKey": geoapify_key}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    
    #Get the lat and lon from the current DF's row
    latitude = hotel_df.loc[index, "Lat"]
    longitude = hotel_df.loc[index, "Lng"]
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{longitude},{latitude},{radius}"
    params["bias"] = f"proximity:{longitude},{latitude}"
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"


    # Make and API request using the params dictionary
    name_address = requests.get(base_url, params=params)
    
    # Convert the API response to JSON format
    name_address_json = name_address.json()
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address_json["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
hohhot - nearest hotel: Yue Jia Budget Hotel (foreigners approved)
bredasdorp - nearest hotel: Victoria Hotel
port elizabeth - nearest hotel: Waterford Hotel
jampur - nearest hotel: City Marriage Hall
nelson bay - nearest hotel: No hotel found
bilibino - nearest hotel: No hotel found
sanjwal - nearest hotel: No hotel found
kingscliff - nearest hotel: Kingscliff Beach Hotel
romorantin-lanthenay - nearest hotel: Le Cheval Blanc
sur - nearest hotel: Sur Hotel
lanivtsi - nearest hotel: No hotel found
tosontsengel - nearest hotel: Алтай
abaete - nearest hotel: Mark's Hotel
zadar - nearest hotel: No hotel found
mount isa - nearest hotel: Ibis Styles
mukalla - nearest hotel: فندق الجابري
namibe - nearest hotel: Nelsal Pensao
damavand - nearest hotel: Avishan Hotel


,City,Country,Lat,Lng,Humidity,Hotel Name
32,hohhot,CN,40.8106,111.6522,72,Yue Jia Budget Hotel (foreigners approved)
33,bredasdorp,ZA,-34.5322,20.0403,86,Victoria Hotel
42,port elizabeth,ZA,-33.9180,25.5701,94,Waterford Hotel
101,jampur,PK,29.6441,70.5954,27,City Marriage Hall
119,nelson bay,AU,-32.7167,152.1500,87,No hotel found
128,bilibino,RU,68.0546,166.4372,48,No hotel found
161,sanjwal,PK,33.7608,72.4336,31,No hotel found
187,kingscliff,AU,-28.2667,153.5667,100,Kingscliff Beach Hotel
223,romorantin-lanthenay,FR,47.4167,1.8333,73,Le Cheval Blanc
248,sur,OM,22.5667,59.5289,58,Sur Hotel


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [7]:
%%capture --no-display

# Configure the map plot
map_plot_2 = hotel_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    frame_width = 800,
    frame_height = 600,
    size = "Humidity",
    color="City",
    alpha=.75,
    hover_cols=["Hotel Name", "Country"]
)

# Display the map
map_plot_2

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Hotel Name,Country)